- 전반적인 EDA 후에 데이터 전처리 방향 더 생각해보기
- COVID19에 의해 가장 달라지는 점은 "소비하는 종목"이라고 생각-> 이부분 처리 필요
- '확진자수'라는 잠재변수를 생성하여 적용해 볼 수도 있다고 생각함. 
- 적용할 모델 : CNN

In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/201901-202003.csv")

In [ ]:
data.head()

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4
1,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8
2,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6
3,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5
4,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3


In [ ]:
# 가구유형별 이용금액 확인 
# 큰차이는 없으나, 1인가구의 비중이 매우 적다. 
# 성인자녀 가구의 비중이 매우 높다
df['AMT'].groupby(df['FLC']).sum()

NameError: ignored

In [ ]:
df['AMT'].groupby(df['STD_CLSS_NM']).sum()

STD_CLSS_NM
건강보조식품 소매업                 105126364380
골프장 운영업                    595701968828
과실 및 채소 소매업                618928801283
관광 민예품 및 선물용품 소매업           15129406137
그외 기타 분류안된 오락관련 서비스업           37246490
그외 기타 스포츠시설 운영업              2731915234
그외 기타 종합 소매업              1025691118171
기타 대형 종합 소매업              7328854819388
기타 수상오락 서비스업                 1671308986
기타 외국식 음식점업                623167678948
기타 주점업                     164753620805
기타음식료품위주종합소매업             1569246626978
내항 여객 운송업                   42015061681
마사지업                        20199890964
면세점                        404512192532
버스 운송업                     230236675997
비알콜 음료점업                  1143497863742
빵 및 과자류 소매업                893591674917
서양식 음식점업                  1748358904526
수산물 소매업                    297031773253
슈퍼마켓                      6730329364274
스포츠 및 레크레이션 용품 임대업        1245725426753
여관업                        258904036530
여행사업                       109016697282
욕탕업                        1

In [ ]:
# 연령대에 따른 이용금액 확인 
# 성인자녀를 둔 가구의 경우의 부모의 이용금액이 가장 높다. 
df['AMT'].groupby(df['AGE']).sum()

AGE
10s      156675306573
20s     6980529840748
30s    11519349556472
40s    14914407109379
50s    15822912458541
60s     8765023628354
70s     2137129148939
Name: AMT, dtype: int64

In [ ]:
# 각 지역별 사용금액을 확인한다. 
df['AMT'].groupby(df['CARD_SIDO_NM']).sum()

CARD_SIDO_NM
강원     1589611299919
경기    14201885234869
경남     3485130029864
경북     2767620595248
광주     1705650360934
대구     3020064352604
대전     1128913460716
부산     4491306031541
서울    14677131049031
세종      204045817348
울산     1255213605740
인천     3056247462913
전남     1839760777139
전북     1961230185341
제주     1343582224768
충남     2103197375108
충북     1465437185923
Name: AMT, dtype: int64

In [ ]:
# 년월에서 년도 확인 
def grap_year(data):
  data = str(data)
  return int(data[:4])

In [ ]:
# 년월에서 월 확인 
def grap_month(data):
  data = str(data)
  return int(data[4:])

In [ ]:
# 날짜 처리 
data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis = 1)

In [ ]:
# 이용금액, 이용건수를 나머지 column을 기준으로 groupby한다. 
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [ ]:
df.head()

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,강원,20s,1,1,2019,1,4,311200,4
1,강원,건강보조식품 소매업,강원,20s,1,1,2019,2,3,605000,3
2,강원,건강보조식품 소매업,강원,20s,1,1,2019,6,3,139000,3
3,강원,건강보조식품 소매업,강원,20s,1,1,2019,8,3,27500,3
4,강원,건강보조식품 소매업,강원,20s,1,1,2019,9,3,395500,3


In [ ]:
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [ ]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [ ]:
# '확진자수' 잠재변수 생성 

RandomForest

In [ ]:
model = RandomForestRegressor(n_jobs=-1, random_state=0)
model.fit(train_features, train_target)

In [ ]:
# 예측 탬플릿 형태 
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [ ]:
temp.shape

(1658860, 8)

In [ ]:
# 예측
pred = model.predict(temp)
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('data/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission_1.csv', encoding='utf-8-sig')
submission.head()

CNN
- sequential data이고 지역적인 정보의 저장이 필요할 것이라 판단해서 사용한 모델

In [ ]:
import pandas as pd
%tensorflow_version 1.x
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import keras

In [ ]:
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)

In [ ]:
X_train = train_features.values
y_train = train_target.values

test_all=submission.values #test 셋

In [ ]:
test_all

array([[202004, '강원', '건강보조식품 소매업'],
       [202004, '강원', '골프장 운영업'],
       [202004, '강원', '과실 및 채소 소매업'],
       ...,
       [202007, '충북', '호텔업'],
       [202007, '충북', '화장품 및 방향제 소매업'],
       [202007, '충북', '휴양콘도 운영업']], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=1)

In [ ]:
X_train.shape

(951654, 8)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
#초기화 
keras.initializers.he_normal(seed=None) #randomnormal
clf = Sequential([
    Dense(15, kernel_initializer='normal', input_dim=8, activation='elu'),
    Dense(20, kernel_initializer='normal', activation='elu'),
    Dropout(0.25),
    Dense(25, kernel_initializer='uniform', activation='elu'),
    Dense(1, kernel_initializer='normal', activation='elu'),
])

In [ ]:
clf.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 15)                135       
_________________________________________________________________
dense_14 (Dense)             (None, 20)                320       
_________________________________________________________________
dropout_4 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 25)                525       
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 26        
Total params: 1,006
Trainable params: 1,006
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#학습
clf.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1)

clf.fit(X_train, y_train, batch_size=15, epochs=10, callbacks=[early_stop])

Epoch 1/10
951654/951654 [==============================] - 257s 270us/step - loss: 4.7889 - mse: 4.7889
Epoch 2/10
951654/951654 [==============================] - 255s 268us/step - loss: 4.6783 - mse: 4.6783
Epoch 3/10
951654/951654 [==============================] - 255s 268us/step - loss: 4.6023 - mse: 4.6024
Epoch 4/10
951654/951654 [==============================] - 257s 270us/step - loss: 4.5532 - mse: 4.5532
Epoch 5/10
951654/951654 [==============================] - 258s 271us/step - loss: 4.5072 - mse: 4.5072
Epoch 6/10
951654/951654 [==============================] - 262s 275us/step - loss: 4.4930 - mse: 4.4929
Epoch 7/10
951654/951654 [==============================] - 260s 274us/step - loss: 4.4434 - mse: 4.4434
Epoch 8/10
951654/951654 [==============================] - 256s 269us/step - loss: 4.4233 - mse: 4.4233
Epoch 9/10
951654/951654 [==============================] - 257s 270us/step - loss: 4.4147 - mse: 4.4147
Epoch 10/10
951654/951654 [============================

In [ ]:
# 예측 탬플릿 형태 
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for year in years:
                            for month in months:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=train_features.columns)

In [ ]:
# 평가 
score = clf.evaluate(X_test, y_test, verbose=0)
print(clf.metrics_names)
print(score) #mse

['loss', 'mse']
[4.278836204831127, 4.27883768081665]


In [ ]:
temp.shape

(1394, 3)

In [ ]:
# predict
test_all_reshape = np.reshape(test_all, (test_all.shape[0], 1, test_all.shape[1]))
pred = clf.predict(temp) #제출정답
pred = np.expm1(pred)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [ ]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [ ]:
# 제출 파일 만들기
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission_2.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,122651.0
1,202004,강원,골프장 운영업,144042.0
2,202004,강원,과실 및 채소 소매업,129182.0
3,202004,강원,관광 민예품 및 선물용품 소매업,99740.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,84988.0
